In [35]:
import glob
import random
import sys

# Plot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import pylab as pl


import numpy as np
import pandas as pd
from pandas import set_option
pd.options.mode.chained_assignment = None
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# use DL Multi-layer Perceptron regressor
from sklearn.neural_network import MLPRegressor
from scipy.stats import truncnorm
from sklearn.model_selection import cross_val_score

from ase.io import read
from dscribe.descriptors import CoulombMatrix
from dscribe.descriptors import ACSF

In [42]:
def ML_train(X_tr, y_tr, X_te, y_te):
    rstate = random.randrange(1,999999999,1)
    estimator = MLPRegressor(random_state = rstate, max_iter=500, hidden_layer_sizes=(20,))
    
    train = estimator.fit(X_tr, y_tr)
    pred = estimator.predict(X_te)
    
    # Cross validation 
    cvscore = cross_val_score(estimator, X_tr, y_tr, cv=5)
    score = cvscore.mean()
    print("Score with the entire dataset = %.2f" % score)
    
    print("Evaluate the error on the test data.")
    print("mean absolute error: ", mean_absolute_error(y_te, pred))
    print("mean squared error: ", mean_squared_error(y_te,pred))
    print("r2 score: ", r2_score(y_te, pred))
    return estimator

In [23]:
aN={1:'H',6:'C',7:'N',8:'O',20:'Ca'}

In [37]:
testsize = 0.3
rstate=random.randrange(1,999999999,1)
scaler = StandardScaler()  



# Machine learning function
def MLP_atom(train_data, test_data):
    # to use the same random number in data shuffling for reproducible results
    # rstate = random.randrange(1,999999999,1)
    X_train = train_data[:,:-1]
    y_train = train_data[:,-1]
    
    X_test = test_data[:,:-1]
    y_test = test_data[:,-1]
    
    # Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
    # apply same transformation to test data
    X_test = scaler.transform(X_test)
    
    #X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = testsize, random_state = rstate)
    estimator = ML_train(X_train, y_train, X_test, y_test)
    return estimator

In [5]:
class Structure:
    
    def __init__(self, pdb_path):
        self.name = pdb_path.split('/')[1].split('.pdb')[0]
        self.water = int(self.name.split('.')[2][5:])
        self.system = self.name.split('.')[0]
        self.loop = self.name.split('.')[1]
        self.mdframe = self.name.split('.')[3]
        self.feat_atom = dict()
        self.cutAtoms = dict()
        #print("PDB name is ", self.name)
        
    def calc_sym(self):
        '''
        The function takes the structure file and compute the 
        symmetry function for each atom
        output: a list of symmetry functions for each atom. 
                Length of the list = number of atoms
        ''' 
        pdb_file = self.name+'.pdb'
        chg_file = self.name + '.chg'
        structure = read("pdb/"+pdb_file)
        feat_lst=[]
        num=structure.get_global_number_of_atoms()
        atomPos = [x for x in range(0,num)]

        chg=np.genfromtxt("charge/"+chg_file)


        #for r_cut in [x/4. for x in range(4,13,1)]:
         
        # Setting up the ACSF descriptor
        for r_cut in [4.]:
            acsf = ACSF(
                species=["C", "H", "O", "N", "Ca"],
                rcut=r_cut,
                g2_params=[[0.0001,0]],
            )
            feat_lst.append(acsf.create(structure,positions=atomPos,n_jobs=4))

        acsf_ang = ACSF(
            species=["C", "H", "O", "N", "Ca"],
            rcut=3.0,
            g4_params=[[0.0001,0.5,-1]],
        )
        feat_lst.append(acsf_ang.create(structure,positions=atomPos,n_jobs=4))

        feat = np.hstack(tuple(feat_lst))
        dat = np.hstack((feat,chg))
        np.savetxt('sym/sym_'+pdb_file[0:-4]+'.dat', dat, delimiter=',')

        # Skip water here
        # Skip to the last water x 3 rows
        skip_water = -3 * self.water
        # If there is no water, skip none (-1)
        if skip_water == 0:
            skip_water = None
        #print("skip water atoms",skip_water)
        #print(dat.shape)
        #print(dat[:skip_water,:].shape)
        for i in dat[:skip_water,:]:
            ikey = aN[int(i[-2])]
            if ikey in self.feat_atom:
                self.feat_atom[ikey] = np.vstack((self.feat_atom[ikey],i))
            else:
                #print(ikey,"is not in the dictionary, adding ...")
                self.feat_atom[ikey] = i

    
    def calc_nb(self, rcut):
        '''
        The function takes the structure file and compute the 
        neighbouring atoms within a cutoff 
        for the coulombic matrix for each atom
        output: a list of atoms groups (structure) for each atom. 
                Length of the list = number of atoms
        ''' 
        pdb_file = self.name+'.pdb'
        chg_file = self.name + '.chg'
        structure = read("pdb/"+pdb_file)
        num=structure.get_global_number_of_atoms()
        
        chg = np.genfromtxt("charge/"+chg_file)
        
        
        # atom index starts from 0
        # Skip water: num - 3*self.water
        for atom_index in range(num-3*self.water):
            atom = structure[atom_index]
            elem = atom.symbol
            
            if elem not in self.cutAtoms:
                self.cutAtoms[elem] = list()
            
            nb = structure[structure.get_distances(atom_index, range(num)) < rcut]
            self.cutAtoms[elem].append((nb,chg[atom_index,1]))

In [6]:

# Use all the files
loops = glob.glob("pdb/*.pdb")
structures = dict()

# record max number of neighbouring atoms for each element
max_atom = {'C':0, 'N':0, 'O':0, 'H':0, 'Ca':0}

counter = 0
count_total = len(loops)
for loop in loops:
    name = loop.split('/')[1].split('.pdb')[0]
    structures[name]=Structure(loop)
    structures[name].calc_nb(3.5)  

    counter = counter + 1
    if counter % 100 == 0:
        #sys.stdout.write('\r')
        print(counter, "structures out of ", count_total, " processed ...", end = '\r')
        sys.stdout.flush()
    
    for elem in max_atom:
        for atom in structures[name].cutAtoms[elem]:
            n_nb = atom[0].get_global_number_of_atoms()
            if max_atom[elem] < n_nb:
                max_atom[elem] = n_nb

64

In [13]:
cm=dict()
for elem in max_atom:
    cm[elem]=CoulombMatrix(n_atoms_max=max_atom[elem])

def calc_cm(ele, nb):
    cmi = cm[ele].create(nb[0])
    chg = nb[1]
    return np.hstack((cmi[0], chg))

In [14]:
# Split the data into train and data
name=np.array([i for i in structures.keys()])
struct_Train, struct_Test = train_test_split(name, test_size = testsize, random_state = rstate)

In [15]:
max_atom

{'C': 27, 'N': 29, 'O': 29, 'H': 30, 'Ca': 25}

In [16]:
# Put data in the train, test dicts.
# Data sorted as for elements
Train_Data_atom = dict()
for loop_name in struct_Train[:1000]:
    
    for ele in aN.values():
        tmp_cm_ele = list()
        nb_all = structures[loop_name].cutAtoms[ele]
        for nb in nb_all:
            cm_ele_loop = calc_cm(ele, nb)
            tmp_cm_ele.append(cm_ele_loop)
        cm_ele = np.vstack(tmp_cm_ele)
        if ele in Train_Data_atom:
            Train_Data_atom[ele] = np.vstack((Train_Data_atom[ele],cm_ele))
        else:
            Train_Data_atom[ele] = cm_ele
if True:
    Test_Data_atom = dict()
    for loop_name in struct_Test[:400]:
        for ele in aN.values():
            tmp_cm_ele = list()
            nb_all = structures[loop_name].cutAtoms[ele]
            for nb in nb_all:
                cm_ele_loop = calc_cm(ele, nb)
                tmp_cm_ele.append(cm_ele_loop)
            cm_ele = np.vstack(tmp_cm_ele)
            if ele in Test_Data_atom:
                Test_Data_atom[ele] = np.vstack((Test_Data_atom[ele],cm_ele))
            else:
                Test_Data_atom[ele] = cm_ele


In [17]:
Train_Data_atom['Ca'].shape

(1000, 626)

In [43]:
# ML models for each element out of 70% of the data
from sklearn.ensemble import RandomForestRegressor

estimator=dict()
for ele in Train_Data_atom:
    print("Training for atoms according to the element: ", ele)
    estimator[ele] = MLP_atom(Train_Data_atom[ele], Test_Data_atom[ele])
    print("\n")

Training for atoms according to the element:  H
Score with the entire dataset = -428.12
Evaluate the error on the test data.
mean absolute error:  0.10427532648648748
mean squared error:  0.020152710157144666
r2 score:  0.1685807405681572


Training for atoms according to the element:  C
Score with the entire dataset = 0.49
Evaluate the error on the test data.
mean absolute error:  0.25260952520830576
mean squared error:  0.11311334131996766
r2 score:  0.5772945098596047


Training for atoms according to the element:  N
Score with the entire dataset = 0.32
Evaluate the error on the test data.
mean absolute error:  0.2429502181710439
mean squared error:  0.18145383553681396
r2 score:  -0.19471907749487993


Training for atoms according to the element:  O
Score with the entire dataset = 0.27
Evaluate the error on the test data.
mean absolute error:  0.09316065211864705
mean squared error:  0.01671444329850079
r2 score:  0.28247413476478767


Training for atoms according to the element:  

In [ ]:
class MLPlot:
    
    # Edit the font, font size, and axes width
    mpl.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 24
    plt.rcParams['axes.linewidth'] = 3
    
    def __init__(self, MLdata, MLestimator):
        X = MLdata[:,:-2]
        y = MLdata[:,-1]
        #X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = testsize, random_state = rstate)
        #self.qm = y_test
        #self.ml = MLestimator.predict(X_test)
        self.qm = y
        self.ml = MLestimator.predict(X)
        
    # Plot outputs
    def corrplot(self):
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot()

        plt.plot(self.qm, self.ml,'ro',fillstyle='none')
        
        # determine the range
        qm_max=np.max(self.qm)
        qm_min=np.min(self.qm)
        ml_max=np.max(self.ml)
        ml_min=np.min(self.ml)
        sigma=np.std(self.qm)+np.std(self.ml)
        ax_low=min(qm_min,ml_min)-sigma
        ax_up=max(qm_max,ml_max)+sigma
        
        plt.xlim(ax_low,ax_up)
        plt.ylim(ax_low,ax_up)
        plt.plot([ax_low,ax_up],[ax_low,ax_up], '--b')

        plt.xlabel('QM i-RESP charge (e)')
        plt.ylabel('ML charge (e)')
        
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(axis='both', which='major', direction='in', length=14, width=4, color='k')
        ax.tick_params(axis='both', which='minor', direction='in', length=8, width=2, color='k')


        #ax.xaxis.grid(True, which='minor')
        # square figure
        ax.set_aspect('equal', adjustable='box')
        plt.show()
    
    
    def cmplot(self):
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot()
        skip = int(len(self.qm)/1000)
        plt.plot(self.qm[::skip], color='red', marker='o', linewidth=1, label = 'QM i-RESP charge (e)')
        plt.plot(self.ml[::skip], color='blue', marker='o', linewidth=1, label = 'ML charge (e)')
        plt.legend(loc='best')
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(axis='both', which='major', direction='in', length=14, width=4, color='k')
        ax.tick_params(axis='both', which='minor', direction='in', length=8, width=2, color='k')
        
        plt.xlabel('Index')
        plt.ylabel('Atomic charge (e)')
        plt.show()

In [ ]:
# Chopping the data for validation
if False:
    X_train_total_size = X_train.shape[0]
    chunk = 1
    chunk_size = int(X_train_total_size / chunk)
    for i in range(chunk):
        row = 1 + i*chunk_size + chunk_size
        sub_X_train = X_train[0:row,:]
        sub_y_train = y_train[0:row]
        ML_train(sub_X_train,sub_y_train,X_test,y_test)

In [ ]:
Caplot = MLPlot(Data_atom['Ca'], estimator['Ca'])
Caplot.corrplot()
Caplot.cmplot()

In [ ]:
# Now investigate what happens to structures with large number of water molecules
# First collecting the structures.

Data_atom_sub = dict()
for loop_name in struct_Test:
    struct = structures[loop_name]
    for ele in struct.feat_atom:
        tup_key = (ele, struct.water)
        if tup_key in Data_atom_sub:
            Data_atom_sub[tup_key] = np.vstack((Data_atom_sub[tup_key], struct.feat_atom[ele]))
        else:
            Data_atom_sub[tup_key] = struct.feat_atom[ele]

In [ ]:
for group in Data_atom_sub:
    print("\nchecking group of structures with", group[1], "for", group[0], "atoms")
    MLdata = Data_atom_sub[group]
    MLestimator = estimator[group[0]]
    X = MLdata[:,:-2]
    y = MLdata[:,-1]
    pred = MLestimator.predict(X)
    print("Evaluate the error.")
    print("mean absolute error: ", mean_absolute_error(y, pred))
    print("mean squared error: ", mean_squared_error(y,pred))
    print("r2 score: ", r2_score(y, pred))


In [ ]:
# Evaluate distribution of the QM charges in the test set
for ele, feat in Test_Data_atom.items():
    _ = plt.hist(feat[:,-1], bins='auto')
    plt.title("Histogram of " + ele + " atoms in the test set")
    plt.show()

In [ ]:
# Evaluate distribution of the QM charges in the training set
for ele, feat in Train_Data_atom.items():
    _ = plt.hist(feat[:,-1], bins='auto')
    plt.title("Histogram of " + ele + " atoms in the training set")
    plt.show()

In [ ]:
# Evaluate distribution of the QM charges in the test set
# According to number of waters
    
water012 = dict()
water3_8 = dict()

for loop_name in struct_Test:
    feat = structures[loop_name].feat_atom
    if structures[loop_name].water < 'water3':
        for item in feat:
            if item not in water012:
                water012[item] = feat[item]
            else:
                water012[item] = np.vstack((water012[item], feat[item]))
    else:
        for item in feat:
            if item not in water3_8:
                water3_8[item] = feat[item]
            else:
                water3_8[item] = np.vstack((water3_8[item], feat[item]))



In [ ]:
for ele in water012:
    print("\nchecking group of structures with <3 water molecules for", ele, "atoms")
    MLdata = water012[ele]
    MLestimator = estimator[ele]
    X = MLdata[:,:-2]
    y = MLdata[:,-1]
    pred = MLestimator.predict(X)
    print("Evaluate the error.")
    print("mean absolute error: ", mean_absolute_error(y, pred))
    print("mean squared error: ", mean_squared_error(y,pred))
    print("r2 score: ", r2_score(y, pred))


In [ ]:
for ele in water3_8:
    print("\nchecking group of structures with >= 3 water molecules for", ele, "atoms")
    MLdata = water3_8[ele]
    MLestimator = estimator[ele]
    X = MLdata[:,:-2]
    y = MLdata[:,-1]
    pred = MLestimator.predict(X)
    print("Evaluate the error.")
    print("mean absolute error: ", mean_absolute_error(y, pred))
    print("mean squared error: ", mean_squared_error(y,pred))
    print("r2 score: ", r2_score(y, pred))


In [ ]:
plt.figure#(figsize=(5, 10))
fig, axs = plt.subplots(5,2)
cnt = 0
for ele in water012:
    chg = water012[ele][:,-1]
    axs[cnt,0].hist(chg, bins='auto')
#    axs[cnt,0].title("Histogram of " + ele + " atoms with <3 H2O in the test set")
    cnt = cnt + 1
#    plt.show()

cnt = 0
for ele in water3_8:
    chg = water3_8[ele][:,-1]
    axs[cnt,1].hist(chg, bins='auto')
 #   axs[cnt,1].title("Histogram of " + ele + " atoms with >=3 H2O in the test set")
    cnt = cnt + 1
#    plt.show()

In [ ]:
fig.legends